In [2]:
sizes = [len(rec) for rec in SeqIO.parse("hsp65/hsp65.fasta", "fasta")]
names = [x.description for x in SeqIO.parse("hsp65/hsp65.fasta", "fasta")]
names = [x.split('|')[0] for x in names]


In [34]:
with open('hsp65/updated_hsp65.fasta', 'w') as out_file:
    for fasta in SeqIO.parse('hsp65/hsp65.fasta','fasta'):
        SeqIO.write(fasta, out_file, 'fasta')

### Create HSP65 Database

In [64]:
import pandas as pd 
import numpy as np 
from Bio import SeqIO

In [63]:
df = pd.read_table('hsp65/nuccore_result_table.txt')
df.head()

,Name,Length,Accession,GI
0,Mycobacterium timonense strain 5351974 heat,994,EU239792.1,171466780
1,Mycobacterium bouchedurhonense strain 4355387 ...,994,EU239790.1,171466776
2,Mycobacterium marseillense strain 5356591 heat,1462,EU239787.1,171466772
3,Mycobacterium abscessus subsp. bolletii hsp65,401,AB548601.1,315433461
4,Mycobacterium timonense strain 3256799 heat,994,EU239791.1,171466778


In [65]:
### Remove Unverified and unclassified Bacteria
list_names = df['Name']
g = [x.split(' ',2)[0] for x in list_names]
matching = [s == "Mycobacterium" for s in g]
df_filtered = df[matching]
len(df_filtered)

1428

In [66]:
list_names = df_filtered['Name']
g = [x.split(' ',2)[1] for x in list_names]
matching = [s != "sp." for s in g]
df_filtered = df_filtered[matching]
len(df_filtered)

1198

In [67]:
list_names = df_filtered['Name']
g = [x.split(' ',2)[1] for x in list_names]
matching = [s != "massiliense" for s in g]
df_filtered = df_filtered[matching]
len(df_filtered)

1099

In [68]:
### Split String Name
list_names = df_filtered['Name']
g = [x.split(' ',2)[0] for x in list_names]

s = [x.split(' ',2)[1] for x in list_names]

y = list(zip(g,s))
y = [list(x) for x in y]
z = [' '.join(x) for x in y]

In [69]:
df_filtered['Species'] = z
df_filtered.head()

,Name,Length,Accession,GI,Species
0,Mycobacterium timonense strain 5351974 heat,994,EU239792.1,171466780,Mycobacterium timonense
1,Mycobacterium bouchedurhonense strain 4355387 ...,994,EU239790.1,171466776,Mycobacterium bouchedurhonense
2,Mycobacterium marseillense strain 5356591 heat,1462,EU239787.1,171466772,Mycobacterium marseillense
3,Mycobacterium abscessus subsp. bolletii hsp65,401,AB548601.1,315433461,Mycobacterium abscessus
4,Mycobacterium timonense strain 3256799 heat,994,EU239791.1,171466778,Mycobacterium timonense


In [97]:
df2 = df_filtered[['Name','Species']].groupby('Species').agg(['count'])
df2.columns = ['Abundance']
df2.head()

,Abundance
Species,
Mycobacterium abscessus,63
Mycobacterium aemonae,1
Mycobacterium africanum,4
Mycobacterium agri,1
Mycobacterium aichiense,3


In [116]:
bool1 = df_filtered['Name'].str.contains('ATCC')
bool2 = df_filtered['Name'].str.contains('DSM')
bool3 = df_filtered['Name'].str.contains('CCUG')
bool4 = df_filtered['Name'].str.contains('ASIN')
bool5 = df_filtered['Name'].str.contains('QIA')
bool6 = df_filtered['Name'].str.contains('GTC')

df_filtered['Cultures'] = bool1|bool2|bool3|bool4|bool5|bool6
df_filtered['Cultures'] = df_filtered['Cultures'].astype(int)


culture_codes = df_filtered[bool1|bool2|bool3|bool4|bool5|bool6]
len(culture_codes)

252

In [92]:
df2_s = df2[df2['Abundance'] > 2]
df2_s.head()
test = list(df2_s.index)

## Keep everything in this df
df_auto_pick = df_filtered[~df_filtered['Species'].isin(test)]


## Randomly select two species (Keep Cultures)
df_mult = df_filtered[df_filtered['Species'].isin(test)]

In [132]:
df2 = df_mult[['Species', 'Cultures']].groupby('Species').sum()
df2.columns = ['Abundance']
df2


### Multiples with 0
zeros = df2[df2['Abundance'] < 1]
zeros = list(zeros.index)


### Multiples with 1
ones = df2[df2['Abundance'] == 1]
ones = list(ones.index)


### Multiples with 2
twos = df2[df2['Abundance'] == 2]
twos = list(twos.index)

### Multiples with Greater than 2
greater = df2[df2['Abundance'] > 2]
greater = list(greater.index)


['Mycobacterium arosiense',
 'Mycobacterium asiaticum',
 'Mycobacterium avium',
 'Mycobacterium celatum',
 'Mycobacterium chelonae',
 'Mycobacterium fortuitum',
 'Mycobacterium gordonae',
 'Mycobacterium interjectum',
 'Mycobacterium intermedium',
 'Mycobacterium intracellulare',
 'Mycobacterium kansasii',
 'Mycobacterium malmoense',
 'Mycobacterium mucogenicum',
 'Mycobacterium neoaurum',
 'Mycobacterium nonchromogenicum',
 'Mycobacterium parascrofulaceum',
 'Mycobacterium porcinum',
 'Mycobacterium scrofulaceum',
 'Mycobacterium senegalense',
 'Mycobacterium septicum',
 'Mycobacterium shimoidei',
 'Mycobacterium simiae',
 'Mycobacterium smegmatis',
 'Mycobacterium szulgai',
 'Mycobacterium triviale',
 'Mycobacterium xenopi']

In [130]:
## Select Accession of Organism with two cultures 
twos_A = df_mult[df_mult['Species'].isin(twos)]
twos_A = twos_A[twos_A['Cultures'] == 1]
twos_A = list(twos_A['Accession'])

In [ ]:
species_grouping = df_filtered.set_index('Species')
species_grouping.head()


188

In [ ]:
import random

foo = ['a', 'b', 'c', 'd', 'e']
print(random.choice(foo))

In [179]:
writer = pd.ExcelWriter('Counts_of_Species.xlsx')
df2.to_excel(writer, "Sheet2")
writer.save()

In [57]:
accession = list(df_filtered['Accession'])
from Bio import SeqIO

with open('Filtered_hsp65.fasta', 'w') as out_file:
    for fasta in SeqIO.parse('hsp65/sequence.fasta','fasta'):
        name, sequence = fasta.id, fasta.seq
        if name in accession:
            SeqIO.write(fasta, out_file, 'fasta')

 ### Filter Additional Accession Numbers Here
 Filter Accession Numbers Found to Overlap Species Identification

In [58]:
from Bio import SeqIO
from collections import defaultdict
records=list(SeqIO.parse("Filtered_hsp65.fasta", "fasta"))

#### Find Records that Share the Same Sequence

In [59]:
from collections import OrderedDict

d = OrderedDict()
for record in records:
    if record.seq in d:
        d[record.seq].append(record)
    else:
        d[record.seq] = [record]
        
len(d)

743

The following chunk createsa new dictionary that contains sequences that have more than one record, then creates a list of true false values to check if the mulitple sequences share the same species

In [60]:
q = {k: v for k, v in d.iteritems() if len(v)> 1}

keep = []

for seq, record_set in q.iteritems():
    
    my_list = []
    for record in record_set:
        
        my_list.append(record.description.split(' ')[2])
    my_list = list(set(my_list))
    if len(my_list) > 1:
        keep.append(True)
    else:
        keep.append(False)

The following chunk creates a dictionary of sequences with the same sequences, but different annotations

In [61]:
weird = dict()
for i in range(len(keep)):
    if keep[i]==True:
        weird[q.keys()[i]] = q.values()[i]
len(weird)

19

In [62]:
for seq, record_set in weird.iteritems():
    print seq + ': (' + str(len(record_set)) + ')'
    for record in record_set:
        print '    ' + record.description

CGCCAAGGAGATCGAGCTGGAGGACCCGTACGAGAAGATCGGCGCTGAGCTGGTCAAGGAAGTTGCCAAGAAGACCGACGACGTCGCGGGTGACGGCACCACCACCGCCACCGTTCTTGCCCAGGCCCTGGTCAAGGAAGGTCTGCGTAACGTCGCCGCCGGCGCCAACCCGCTCGGCCTGAAGCGCGGCATCGAGAAGGCCGTCGAGAAGGTCACCGAGACGCTGCTGAAGAGCGCCAAGGAGGTCGAGACCAAGGAGCAGATCGCGGCCACGGCCGGTATCTCCGCGGGCGACCAGTCCATCGGCGACCTGATCGCCGAGGCCATGGACAAGGTTGGCAACGAGGGTGTCATCACCGTCGAGGAGTCCAACACCTTCGGCCTGCAGCTGGAGCTCACCG: (5)
    JF491290.1 Mycobacterium abscessus ATCC 19977 GroEL (hsp65) gene, partial cds
    AB548598.1 Mycobacterium abscessus hsp65 gene for heat shock protein 65, partial cds, strain: JCM 13569
    GU362527.1 Mycobacterium abscessus strain USFLJA0007 GroEL (hsp65) gene, partial cds
    GU362521.1 Mycobacterium abscessus strain USFLJA0001 GroEL (hsp65) gene, partial cds
    AJ310215.1 Mycobacterium chelonae subsp. abscessus hsp65 gene for type I 65 KDa heat shock protein
GAGGACCCGTACGAGAAGATCGGCGCTGAGCTCGTCAAAGAGGTCGCCAAGAAGACTGACGACGTCGCGGGCGACGGCACCACCACCGCCACCGTTCTGGCCCAGGCCCTGGTTCGCGAAGG

In [145]:
count = 0
for seq, record_set in weird.iteritems():
    for record in record_set:
        count += 1
count

214

In [151]:
accession = list(df_filtered['Accession'])

In [33]:
accession = list(df_filtered['Accession'])
from Bio import SeqIO

with open('Filtered_hsp65.fasta', 'w') as out_file:
    for fasta in SeqIO.parse('hsp65/sequence.fasta','fasta'):
        name, sequence = fasta.id, fasta.seq
        if name in accession:
            SeqIO.write(fasta, out_file, 'fasta')

remove TB/ remove sp. 
massillense no longer a species (check date)
ATCC/DSM/CCUG/ASIN/QIA/GTC



